https://github.com/dotnet/try/issues/688

In [ ]:
#r "nuget:Microsoft.Data.Analysis"
open Microsoft.Data.Analysis

In [ ]:
#r @"TakensTheorem.Core.dll"

In [ ]:
open System
open TakensTheorem.Core
open TakensTheorem.Core.ZipHelper
open TakensTheorem.Core.DataFrameColumnOperators

In [ ]:
let path = @"..\Data\historical-hourly-weather-data.zip"

In [ ]:
let loader = new ZippedCsvLoader(path)

In [ ]:
let cityTable = loader.ToDataFrame "city_attributes.csv"
let temperatureDF = loader.ToDataFrame "temperature.csv"
let indexName = temperatureDF.Columns.[0].Name

In [ ]:
temperatureDF.[indexName] <- temperatureDF.[indexName]
                             |> Seq.cast<string>
                             |> Seq.map (DateTime.Parse)
                             |> DataFrameColumn.FromValues indexName

In [ ]:
let start = DateTime(2015, 6, 22)
let stop = DateTime(2015, 8, 31)

(* 
    temperatureDF.["datetime"].ElementwiseGreaterThanOrEqual<DateTime> (start)
    don't work for no dates yet
*)
    
let dateFilterColumn = 
    !> temperatureDF.[indexName]
    |> DataFrameColumn<DateTime>.CreateFilter (fun dt -> dt.HasValue && dt.Value >= start && dt.Value <= stop)
    
let weatherDataMontreal = 
    temperatureDF.Filter(dateFilterColumn).["Montreal"]

In [ ]:
temperatureDF |> DataFrame.Length
//weatherDataMontreal.Filter

In [ ]:
// index unavailable at column level

!> weatherDataMontreal 
|> DataFrameColumn<float>.Plot

In [ ]:
(* subtracting null same as subtracting zero
 * in pandas both become NaN
 * therefore dropping na's after the fact
 * does not trim the column
 *)
let lowPassFilterSignalCol = weatherDataMontreal.Clone()

weatherDataMontreal
|> DataFrameColumn.Rolling<float> 24 (Seq.average)
|> Seq.iteri (fun i value -> lowPassFilterSignalCol.[int64 (i+12)] <- Nullable value)

let weatherDataMontrealFiltered = 
    !> (weatherDataMontreal - lowPassFilterSignalCol)
    |> DataFrameColumn.Trim 0.0

weatherDataMontrealFiltered
|> DataFrameColumn<float>.Plot

In [ ]:
let embeddedWeather = 
    weatherDataMontrealFiltered
    |> DataFrameColumn.TakensEmbedding<float> 5 2    
    |> Array.ofSeq
   
Chart.Plot(Scatter(x=embeddedWeather.[0], y=embeddedWeather.[1]))

In [ ]:
let embeddedWeather3 =
    weatherDataMontrealFiltered
    |> DataFrameColumn.TakensEmbedding<float> 6 3
    |> Array.ofSeq

Chart.Plot(Scatter3d(x=embeddedWeather3.[0], y=embeddedWeather3.[1], z=embeddedWeather3.[2], mode="lines"))

In [ ]:
(* No drop nulls for columns *)

let k = {|
    l0 = temperatureDF.DropNulls(DropNullOptions.Any).Rows.Count
    ln = temperatureDF.Rows.Count
    l1 = weatherDataMontreal.Length
    l2 = DataFrame(weatherDataMontreal).DropNulls(DropNullOptions.Any).Rows.Count
    l3 = 
        !> weatherDataMontreal 
        |> DataFrameColumn<float>.Values
        |> Seq.filter (fun value -> value <> Nullable()) 
        |> Seq.length |> int64

    l4 = lowPassFilterSignalCol |> DataFrameColumn.Length    |}  
{| k with diff = k.l3 - k.l4 |} //l0,ln,l1,l2,l3,l4, l3-l4

In [ ]:
let inline plot' (data: ^a[]) = Chart.Plot(Scatter(y=data ))
let inline plot'' (data: ^a[][]) = Chart.Plot(Scatter(x=data.[0], y=data.[1]))
let inline plot''' (data: ^a[][]) = Chart.Plot(Scatter3d(x=data.[0], y=data.[1],z=data.[2]))

In [ ]:
%%time

[|for i in 1L..20L do 
    DataFrameColumn.MutualInformation i 16 weatherDataMontrealFiltered|]
|> plot'

https://github.com/fslaborg/XPlot/issues/49

In [ ]:

let layout = 
    Layout(
        title = "Heatmap and Time Series Plot beside each other",
        xaxis = Xaxis(),
        yaxis = Yaxis(),
        xaxis2 = Xaxis(),
        yaxis2 = Yaxis()
    )

let embeddedWeather2d_delay5 = DataFrameColumn.TakensEmbedding 5 2 weatherDataMontrealFiltered |> Array.ofSeq
let embeddedWeather2d_delay1 = DataFrameColumn.TakensEmbedding 1 2 weatherDataMontrealFiltered |> Array.ofSeq

// [    Scatter(x=embeddedWeather2d_delay5.[0], y=embeddedWeather2d_delay5.[1],  xaxis="x2", yaxis="y2")
//      Scatter(x=embeddedWeather2d_delay1.[0], y=embeddedWeather2d_delay1.[1],  xaxis="x1", yaxis="y1") ]
// |> Chart.Plot
// |> Chart.WithLayout layout

In [ ]:
plot'' embeddedWeather2d_delay5

In [ ]:
plot'' embeddedWeather2d_delay1